In [1]:
!pip install groq
import os
from groq import Groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 1.9 MB/s eta 0:00:00


In [2]:
from getpass import getpass

api_key = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [3]:
client = Groq(api_key=api_key)


In [4]:
history = []

def add_message(role, content):
    history.append({"role": role, "content": content})

def truncate_by_turns(n):
    return history[-n:]

def truncate_by_length(max_chars):
    total, result = 0, []
    for msg in reversed(history):
        total += len(msg["content"])
        if total > max_chars:
            break
        result.insert(0, msg)
    return result


In [5]:
def summarize_history(cur_history):
    summary_prompt = [
        {"role": "system", "content": "You are a helpful assistant. Summarize the following conversation in 5 sentences."},
        {"role": "user", "content": "\n".join([m["role"] + ": " + m["content"] for m in cur_history])}
    ]
    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=summary_prompt,
        max_tokens=300,
    )
    return response.choices[0].message.content


In [6]:
def periodic_summary(k=3):
    if len(history) % k == 0 and len(history) > 0:
        summary = summarize_history(history)
        history.clear()
        history.append({"role": "system", "content": "Conversation summary: " + summary})




In [7]:
history.clear()

add_message("user", "Can you help me prepare for a data analyst interview?")
add_message("assistant", "Sure! What topics should we focus on?")
add_message("user", "I want to practice SQL and Python questions.")
add_message("assistant", "Great! For SQL, do you want SELECT queries or joins?")
add_message("user", "Both would be helpful.")
add_message("assistant", "Okay, I'll prepare some examples.")
print("Last 2 turns:", truncate_by_turns(2))
print("Truncated to 40 chars:", truncate_by_length(40))
periodic_summary(k=3)
print("Current: ", history)


Last 2 turns: [{'role': 'user', 'content': 'Both would be helpful.'}, {'role': 'assistant', 'content': "Okay, I'll prepare some examples."}]
Truncated to 40 chars: [{'role': 'assistant', 'content': "Okay, I'll prepare some examples."}]
Current:  [{'role': 'system', 'content': 'Conversation summary: Here is a summary of the conversation in 5 sentences:\n\nA user asked for help preparing for a data analyst interview. The user expressed interest in practicing SQL and Python questions. The assistant offered to help and asked which SQL topics to focus on. The user requested practice with both SELECT queries and joins in SQL. The assistant agreed to prepare examples to help the user practice.'}]


In [8]:
schema = {
  "type": "object",
  "properties": {
    "name": {"type": "string"},
    "email": {"type": "string"},
    "phone": {"type": "string"},
    "location": {"type": "string"},
    "age": {
  "anyOf": [
    {"type": "integer"},
    {"type": "string", "pattern": "^[0-9]+$"}
  ]
}

  },
  "required": ["name", "email", "phone", "location", "age"]
}

tools = [
  {
    "type": "function",
    "function": {
      "name": "extract_user_info",
      "description": "Extracts user info from chat.",
      "parameters": schema
    }
  }
]


In [9]:
def classify_user_info(chat_content):
    messages = [
        {"role": "system", "content": "Extract user info from the following chat."},
        {"role": "user", "content": chat_content}
    ]
    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=300,
    )
    arguments = response.choices[0].message.tool_calls[0].function.arguments

    import json
    info = json.loads(arguments)
    if 'age' in info and isinstance(info['age'], str):
        try:
            info['age'] = int(info['age'])
        except ValueError:
            pass

    desired_order = ["name", "email", "phone", "location", "age"]
    ordered_info = {k: info[k] for k in desired_order if k in info}

    return ordered_info


In [10]:
samples = [
    "Hi, I'm Alice. My email is alice@example.com, phone is 1234567890. I live in Mumbai, and I'm 23.",
    "Hey, Bob here. Contact: bob@mail.com, +911234567890, Delhi, 32 years old.",
    "Carol's details: carol@email.com, 9988776655, Bangalore, 27.",
]
for s in samples:
    print(classify_user_info(s))


{'name': 'Alice', 'email': 'alice@example.com', 'phone': '1234567890', 'location': 'Mumbai', 'age': 23}
{'name': 'Bob', 'email': 'bob@mail.com', 'phone': '+911234567890', 'location': 'Delhi', 'age': 32}
{'name': 'Carol', 'email': 'carol@email.com', 'phone': '9988776655', 'location': 'Bangalore', 'age': 27}
